# Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=605976299c11c1cd61875ebdf4ffd9f8af024e60bdd815d84551c122cc749dbe
  St

In [ ]:
import pandas as pd
import ast
import numpy as np
from collections import Counter
from transformers import DataCollatorForTokenClassification
import datasets
from datasets import Dataset
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
from transformers import RobertaForTokenClassification, RobertaTokenizer

# huggingface's datasets library
# from datasets import load_dataset

# the tqdm library used to show the iteration progress
import tqdm
tqdmn = tqdm.notebook.tqdm

from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric

# Load Model

In [ ]:
roberta_version = "urduhack/roberta-urdu-small"
tokenizer = RobertaTokenizer.from_pretrained(roberta_version)

In [ ]:
from evaluate import load

metric2 = load("seqeval")
label_list2= [[0], [1], [2], [3]]

# metric2 = load_metric("seqeval")
def compute_metrics2(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list2[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list2[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric2.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


data_collator = DataCollatorForTokenClassification(tokenizer)
label_all_tokens = True
task = "ner"

In [ ]:
load_path = '/content/drive/MyDrive/NER_Saved_Models/Urdu_MK_PUCIT_NER2_fold10'
modelload = RobertaForTokenClassification.from_pretrained(load_path)

# or
# load_path2 = '/content/drive/MyDrive/NER_Saved_Models/save_pretrained/Urdu_MK_PUCIT_NER2_trainer1_1'
# modelload2 = RobertaForTokenClassification.from_pretrained(load_path2)

In [ ]:
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

# from transformers import AdamW
# optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

### Training Args 1_1 ###
training_args1_1 = TrainingArguments(
    output_dir="./fine_tune_bert_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps = 1000,
    load_best_model_at_end=True,
    save_strategy='epoch'
)

### Trainer 1_1 ###
trainer = Trainer(
    model=modelload,
    args=training_args1_1,
    # train_dataset=train,
    # eval_dataset=validation,

    tokenizer=tokenizer,
    callbacks=[early_stopping],
    data_collator=data_collator,
    compute_metrics=compute_metrics2
)


# News Data & Pre-processing

## Load News Data

### Load Subsets

In [ ]:
filename = "Nov_Sub3of3.csv"
path = f"/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/{filename}"

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLaterMonday/missing_data.csv", encoding="utf-8", lineterminator='\n')

for column in df.columns:
  if 'Unnamed' in column:
    del df[column]
  if 'index' in column:
    del df[column]

print(df.columns)
print(df.shape)
# df.head(2)

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment'],
      dtype='object')
(4158, 16)


##### Merge All Subsets

In [ ]:
import pandas as pd
path1 = f"/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/Merged_Subsets.csv"
path2 = f"/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/Merged_Subsets2.csv"
path3 = f"/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/Merged_Subsets3.csv"
d1 = pd.read_csv(path1, encoding="utf-8", lineterminator='\n')
d2 = pd.read_csv(path2, encoding="utf-8", lineterminator='\n')
d3 = pd.read_csv(path3, encoding="utf-8", lineterminator='\n')

In [ ]:
data = pd.concat([d1, d2, d3], ignore_index=True)

data = data.drop_duplicates()
data.reset_index(inplace=True, drop=True)

print(data.columns)
print(data.shape)
data.head(2)

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc'],
      dtype='object')
(104669, 20)


,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized,NER_Persons,NER_Orgs,NER_Loc
0,کراچی کو مجرموں کا شہر نہیں بننے دیں گے: کامرا...,2023-04-15,https://www.nawaiwaqt.com.pk/15-Apr-2023/1699923,گورنر سندھ کامران ٹیسوری نے کہا ہے کہ کراچی کو...,Nawaiwaqt,جرم و سزا,کراچی کو مجرموں کا شہر نہیں بننے دیں گے: کامرا...,کراچی کو مجرموں کا شہر نہیں بننے دیں گے کامران...,کراچی مجرموں شہر بننے گے کامران ٹیسوری گورنر س...,کراچی مجرموں شہر بننے گے: کامران ٹیسوری گورنر ...,کراچی مجرموں شہر بننے گے کامران ٹیسوری گورنر س...,crime,NotAssigned,"['ٹیسوری', 'شہر', 'کامران', 'سحری', 'ٹوٹے', 'م...",40737,1.0,"['کراچی', 'کو', 'مجرموں', 'کا', 'شہر', 'نہیں',...",['کامران'],[],"['روڈ', 'کراچی']"
1,کراچی کو مجرموں کا شہر نہیں بننے دیں گے: کامرا...,2023-04-15,https://www.nawaiwaqt.com.pk/15-Apr-2023/1699923,گورنر سندھ کامران ٹیسوری نے کہا ہے کہ کراچی کو...,Nawaiwaqt,قومی,کراچی کو مجرموں کا شہر نہیں بننے دیں گے: کامرا...,کراچی کو مجرموں کا شہر نہیں بننے دیں گے کامران...,کراچی مجرموں شہر بننے گے کامران ٹیسوری گورنر س...,کراچی مجرموں شہر بننے گے: کامران ٹیسوری گورنر ...,کراچی مجرموں شہر بننے گے کامران ٹیسوری گورنر س...,pakistan,NotAssigned,"['ٹیسوری', 'شہر', 'کامران', 'سحری', 'ٹوٹے', 'م...",40738,1.0,"['کراچی', 'کو', 'مجرموں', 'کا', 'شہر', 'نہیں',...",['کامران'],[],"['روڈ', 'کراچی']"


In [ ]:
df.columns

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC'],
      dtype='object')

In [ ]:
subsetcols = ['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC']

In [ ]:
data2 = data.drop_duplicates(subset=subsetcols)
data2.reset_index(inplace=True, drop=True)

print(data2.columns)
print(data2.shape)

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc'],
      dtype='object')
(99011, 20)


In [ ]:
data2 = data2[subsetcols]

In [ ]:
df3 = df[~df.isin(data2.to_dict(orient='list')).all(axis=1)].drop_duplicates()
df3.shape

(4158, 14)

In [ ]:
df3.head(2)

,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC
335,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,2023-05-31,https://www.bbc.com/urdu/articles/c0j5ze7l39no,پاکستان تحریک انصاف کے چئیرمین عمران خان کے خل...,BBC,پاکستان,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,عمران خان کی عدالت میں پیشی کا احوال جج تھری و...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,عمران خان عدالت پیشی احوال: ’جج تھری ورکنگ ڈیز...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,pakistan,NotAssigned,"['عمران', 'عدالت', 'خان', 'ضمانت', 'وکلا', 'مق..."
336,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,2023-05-30,https://www.bbc.com/urdu/articles/cd1zdj4evl3o,چیئرمین تحریک انصاف عمران خان کے خلاف کم از کم...,BBC,پاکستان,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,pakistan,NotAssigned,"['دفعات', 'کارکنان', 'درج', 'آر', 'ایف', 'عمرا..."


In [ ]:
df3['DateTime'].unique()

array(['2023-05-31', '2023-05-30', '2023-05-29', '2023-05-28',
       '2023-05-27', '2023-05-25', '2023-05-24', '2023-05-23',
       '2023-05-22', '2023-05-21', '2023-05-17', '2023-05-16',
       '2023-05-14', '2023-05-19', '2023-05-15', '2023-05-12',
       '2023-05-11', '2023-05-07', '2023-05-06', '2023-05-05',
       '2023-05-01', '2023-05-03', '2023-05-02', '2023-05-08',
       '2023-05-04', '2023-05-09', '2023-05-10', '2023-05-13',
       '2023-05-18', '2023-05-20', '2023-05-26'], dtype=object)

In [ ]:
df3.to_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/missing_data.csv", index=False)

In [ ]:
data2.to_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/NewsDataWithSentsNER.csv", index=False)

In [ ]:
is_subset = data2.isin(df.to_dict(orient='list')).all(axis=1).all()

if is_subset:
    print("data2 is a subset of df")
else:
    print("data2 is not a subset of df")

data2 is a subset of df


In [ ]:
if df.shape[0]==data.shape[0]:
  print("YASS")

In [ ]:
# Define the folder path containing your CSV files in Google Drive
folder_path = '/content/drive/MyDrive/Datasets/SubsetsToDeleteLater'

import os

# List CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

import pandas as pd

# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Loop through each CSV file and concatenate them
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    data = pd.read_csv(file_path)
    merged_data = pd.concat([merged_data, data], ignore_index=True)

# Optionally, save the merged data to a new CSV file
merged_data.to_csv('/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/Merged_Subsets3.csv', index=False)

# Display the first few rows of the merged data
print(merged_data.shape)
merged_data.head()


(50053, 20)


,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized,NER_Persons,NER_Orgs,NER_Loc
0,نوال سعید کا نور حسن سے اظہار محبت,2023-04-26,https://jang.com.pk/news/1219120,اداکارہ نوال سعید نے ساتھی اداکار نور حسن کے ل...,Jang,انٹرٹینمنٹ,نوال سعید کا نور حسن سے اظہار محبت اداکارہ نوا...,نوال سعید کا نور حسن سے اظہار محبت اداکارہ نوا...,نوال سعید نور حسن اظہار محبت اداکارہ نوال سعید...,نوال سعید نور حسن اظہار محبت اداکارہ نوال سعید...,نوال سعید نور حسن اظہار محبت اداکارہ نوال سعید...,entertainment,NotAssigned,"['نوال', 'نور', 'حسن', 'سعید', 'تعریف', 'پیار'...",18506,2,"['نوال', 'سعید', 'کا', 'نور', 'حسن', 'سے', 'اظ...","['حسن', 'نوال سعید', 'نوال', 'سعید']",[],['اسلام']
1,ہیری پوٹر سیریز میں مرکزی کردار ادا کرنے والے ...,2023-04-26,https://jang.com.pk/news/1219109,ہیری پوٹر سیریز کی فلموں میں ہیری پوٹر کا مرکز...,Jang,انٹرٹینمنٹ,ہیری پوٹر سیریز میں مرکزی کردار ادا کرنے والے ...,ہیری پوٹر سیریز میں مرکزی کردار ادا کرنے والے ...,ہیری پوٹر سیریز مرکزی کردار ادا ڈینیل ریڈکلف و...,ہیری پوٹر سیریز مرکزی کردار ادا ڈینیل ریڈکلف و...,ہیری پوٹر سیریز مرکزی کردار ادا ڈینیل ریڈکلف و...,entertainment,NotAssigned,"['ریڈکلف', 'ڈینیل', 'پوٹر', 'ہیری', 'ڈارک', 'ا...",18507,1,"['ہیری', 'پوٹر', 'سیریز', 'میں', 'مرکزی', 'کرد...","['ڈارک', 'ڈینیل ریڈکلف', 'ایرن', 'ڈینیل', 'ایر...",[],[]
2,شہروز سبزواری ’وائٹ کرولا گینگ‘ کے ہاتھوں اغوا...,2023-04-26,https://jang.com.pk/news/1219106,پاکستانی سینئر، معروف اداکار بہروز سبزواری کے ...,Jang,انٹرٹینمنٹ,شہروز سبزواری ’وائٹ کرولا گینگ‘ کے ہاتھوں اغوا...,شہروز سبزواری وائٹ کرولا گینگ کے ہاتھوں اغوا ہ...,شہروز سبزواری وائٹ کرولا گینگ ہاتھوں اغوا پاکس...,شہروز سبزواری ’وائٹ کرولا گینگ‘ ہاتھوں اغوا پا...,شہروز سبزواری وائٹ کرولا گینگ ہاتھوں اغوا پاکس...,entertainment,NotAssigned,"['سبزواری', 'شہروز', 'بہروز', 'گینگ', 'اغوا', ...",18508,1,"['شہروز', 'سبزواری', '’وائٹ', 'کرولا', 'گینگ',...","['سنائی', 'شہروز']",[],['کراچی']
3,کینیڈین گلوکارہ نے مصنوعی ذہانت کے ذریعے بنائ...,2023-04-26,https://jang.com.pk/news/1219104,کینیڈین گلوکارہ گرائمز نے مصنوعی ذہانت کے پروگ...,Jang,انٹرٹینمنٹ,کینیڈین گلوکارہ نے مصنوعی ذہانت کے ذریعے بنائ...,کینیڈین گلوکارہ نے مصنوعی ذہانت کے ذریعے بنائے...,کینیڈین گلوکارہ مصنوعی ذہانت بنائے گانوں آواز ...,کینیڈین گلوکارہ مصنوعی ذہانت بنائے گانوں آواز...,کینیڈین گلوکارہ مصنوعی ذہانت بنائے گانوں آواز ...,entertainment,NotAssigned,"['گلوکارہ', 'گرائمز', 'ذہانت', 'مصنوعی', 'گانو...",18509,1,"['کینیڈین', 'گلوکارہ', 'نے', 'مصنوعی', 'ذہانت'...","['ٹوئٹر', 'گرل', 'ایلون', 'سامنا', 'ڈریک']",[],['کینیڈا']
4,شاہ رخ خان بیٹے آریان کے برانڈ کیلئے ایمبیسیڈر...,2023-04-26,https://jang.com.pk/news/1219103,بالی ووڈ کنگ شاہ رخ خان بیٹے آریان خان کے کپڑو...,Jang,انٹرٹینمنٹ,شاہ رخ خان بیٹے آریان کے برانڈ کیلئے ایمبیسیڈر...,شاہ رخ خان بیٹے آریان کے برانڈ کیلئے ایمبیسیڈر...,شاہ رخ خان بیٹے آریان برانڈ کیلئے ایمبیسیڈر با...,شاہ رخ خان بیٹے آریان برانڈ کیلئے ایمبیسیڈر با...,شاہ رخ خان بیٹے آریان برانڈ کیلئے ایمبیسیڈر با...,entertainment,NotAssigned,"['آریان', 'برانڈ', 'رخ', 'خان', 'تشہیری', 'ایم...",18510,1,"['شاہ', 'رخ', 'خان', 'بیٹے', 'آریان', 'کے', 'ب...","['دشواری', 'آریان', 'نقش', 'آریان خان', 'خان',...",['یونیورسٹی'],['امریکا']


### Load Main

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/NewsData2022_2023-06-15.csv", encoding="utf-8", lineterminator='\n')

for column in df.columns:
  if 'Unnamed' in column:
    del df[column]
  if 'index' in column:
    del df[column]


print(df.columns)
print(df.shape)
df.head(2)

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC'],
      dtype='object')
(103169, 14)


,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC
0,امریکی وزیر خارجہ کے نگراں پاکستانی وزیراعظم ک...,2023-08-18,https://www.bbc.com/urdu/articles/c72j665dv0zo,امریکی وزیر خارجہ انٹونی بلنکن کا ایکس (ٹوئٹر)...,BBC,پاکستان,امریکی وزیر خارجہ کے نگراں پاکستانی وزیراعظم ک...,امریکی وزیر خارجہ کے نگراں پاکستانی وزیراعظم ک...,امریکی وزیر خارجہ نگراں پاکستانی وزیراعظم تہنی...,امریکی وزیر خارجہ نگراں پاکستانی وزیراعظم تہنی...,امریکی وزیر خارجہ نگراں پاکستانی وزیراعظم تہنی...,pakistan,NotAssigned,"['کمیونٹی', 'نوٹس', 'ایکس', 'قارئین', 'کانٹریب..."
1,جڑانوالہ میں مسیحی برداری پر حملے: ’سوشل میڈیا...,2023-08-18,https://www.bbc.com/urdu/articles/c72jlj8rendo,تھانہ سٹی جڑانوالہ نے انسداد دہشتگردی ایکٹ اور...,BBC,پاکستان,جڑانوالہ میں مسیحی برداری پر حملے: ’سوشل میڈیا...,جڑانوالہ میں مسیحی برداری پر حملے سوشل میڈیا پ...,جڑانوالہ مسیحی برداری حملے سوشل میڈیا اقلیتوں ...,جڑانوالہ مسیحی برداری حملے: ’سوشل میڈیا اقلیتو...,جڑانوالہ مسیحی برداری حملے سوشل میڈیا اقلیتوں ...,pakistan,NotAssigned,"['مسیحی', 'جڑانوالہ', 'اقلیتوں', 'چرچ', 'مذہبی..."


In [ ]:
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)

df.shape

(103169, 14)

## Pre-processing

In [ ]:
df['Combined'] = df['Combined'].replace('nbsp','')

In [ ]:
def add_space_before_punctuation(urdu_text):
    # PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`‘’۔،{|}~"""
    PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`‘’۔،{|}~؛،؟‘’“”()[]{}<>?!-…—–:;،۔"""
    result = []

    for char in urdu_text:
        # Check if the character is a punctuation mark and there's no space before it
        if char in PUNCTUATION and (not result or result[-1] != ' '):
            result.append(' ')

        result.append(char)

    return ''.join(result)


df['CC_tokenized'] = df['Combined'].apply(add_space_before_punctuation)
df['CC_tokenized'] = df['CC_tokenized'].apply(lambda x: x.split())
df.head(2)

,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized
0,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,2023-05-31,https://www.bbc.com/urdu/articles/c0j5ze7l39no,پاکستان تحریک انصاف کے چئیرمین عمران خان کے خل...,BBC,پاکستان,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,عمران خان کی عدالت میں پیشی کا احوال جج تھری و...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,عمران خان عدالت پیشی احوال: ’جج تھری ورکنگ ڈیز...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,pakistan,NotAssigned,"['عمران', 'عدالت', 'خان', 'ضمانت', 'وکلا', 'مق...",1,1,"[عمران, خان, کی, عدالت, میں, پیشی, کا, احوال, ..."
1,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,2023-05-30,https://www.bbc.com/urdu/articles/cd1zdj4evl3o,چیئرمین تحریک انصاف عمران خان کے خلاف کم از کم...,BBC,پاکستان,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,pakistan,NotAssigned,"['دفعات', 'کارکنان', 'درج', 'آر', 'ایف', 'عمرا...",2,1,"[نو, مئی, کے, ہنگاموں, پر, عمران, خان, کے, خلا..."


In [ ]:
for index, item in enumerate(df['CC_tokenized']):
  new = [[value] for value in item]
  df.loc[index, 'CC_tokenized'] = new

df.head(1)

,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized
0,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,2023-05-31,https://www.bbc.com/urdu/articles/c0j5ze7l39no,پاکستان تحریک انصاف کے چئیرمین عمران خان کے خل...,BBC,پاکستان,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,عمران خان کی عدالت میں پیشی کا احوال جج تھری و...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,عمران خان عدالت پیشی احوال: ’جج تھری ورکنگ ڈیز...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,pakistan,NotAssigned,"['عمران', 'عدالت', 'خان', 'ضمانت', 'وکلا', 'مق...",1,1,"[عمران, خان, کی, عدالت, میں, پیشی, کا, احوال, ..."


### Split into IDs and Tokens

In [ ]:
df_CC = pd.DataFrame(columns=['id', 'token'])

CCids = [] # list to store ids
# newsids = [] # list to store ids
CClist = [] # list to store Headline tokens
# Nlist = [] # list to store News tokens

for index, id in enumerate(df['id']):
  CC = df['CC_tokenized'][index]
  if 'error' in CC:
    print('FOUND ERROR')
    CCids.append(id)
    CClist.append('error')
  elif 'error' not in CC:
    for i, token in enumerate(CC):
      CCids.append(id)
      CClist.append(token)


df_CC['id'] = CCids
df_CC['token'] =  CClist

print(df_CC.shape)
df_CC.head()

(982482, 2)


,id,token
0,1,عمران
1,1,خان
2,1,کی
3,1,عدالت
4,1,میں


## Convert to DatasetDict Format

In [ ]:
%%time
combined_dataset = Dataset.from_dict(df_CC)
dataset = datasets.DatasetDict({"Combined": combined_dataset})
dataset

CPU times: user 262 ms, sys: 48.1 ms, total: 310 ms
Wall time: 327 ms


DatasetDict({
    Combined: Dataset({
        features: ['id', 'token'],
        num_rows: 982482
    })
})

In [ ]:
dataset.shape

{'Combined': (982482, 2)}

## Get Encodings

In [ ]:
def add_encodings(examples):

    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True, padding='max_length', max_length=5)
    # tokenized_inputs["labels"] = examples['tags']
    return tokenized_inputs


In [ ]:
%%time
dataset = dataset.map(add_encodings, remove_columns=['token'])

dataset.set_format(type='torch')

dataset

Map:   0%|          | 0/982482 [00:00<?, ? examples/s]

CPU times: user 3min 57s, sys: 2.21 s, total: 3min 59s
Wall time: 4min 17s


DatasetDict({
    Combined: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 982482
    })
})

In [ ]:
dataset.shape

{'Combined': (982482, 3)}

# Get Predictions

In [ ]:
%%time
# predictions1, labels1, metrics1 = trainer.predict(dataset['Combined'], metric_key_prefix="predict")
predictions1 = trainer.predict(dataset['Combined'])

CPU times: user 5min 22s, sys: 1.64 s, total: 5min 23s
Wall time: 5min 30s


# Get all the NERs in proper format

In [ ]:
%%time
outputCC = torch.tensor([predictions1[0]])
# outputN = torch.tensor([predictions2])
index2tag = {0: 'Location', 1: 'Other', 2: 'Organization', 3: 'Person'}
predicted_labelsCC = torch.argmax(outputCC, dim=-1)

from collections import Counter
preds = []
for i in predicted_labelsCC[0]:
  # print(i)
  counted_numbers = Counter(i.tolist())
  most_common_number = counted_numbers.most_common(1)[0][0]
  # print(most_common_number)
  preds.append(most_common_number)


print(len(preds))

df_CC['pred'] = preds
index2tag = {0: 'Location', 1: 'Other', 2: 'Organization', 3: 'Person'}
df_CC['tag'] = df_CC['pred'].map(index2tag)
df_CC.head()

<timed exec>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)


982482
CPU times: user 20.3 s, sys: 461 ms, total: 20.7 s
Wall time: 20.9 s


,id,token,pred,tag
0,1,عمران,3,Person
1,1,خان,3,Person
2,1,کی,1,Other
3,1,عدالت,1,Other
4,1,میں,1,Other


In [ ]:
df2 = pd.DataFrame(columns=['id', 'NER_Persons', 'NER_Orgs', 'NER_Loc'])
# df['NER_Persons'] = ''
# df['NER_Orgs'] = ''
# df['NER_Loc'] = ''
for idx, row in df.iterrows():
    ID = row['id']
    # ID = 2
    sub = df_CC[df_CC['id'] == ID]
    # print(sub.shape)
    pers = []
    orgs = []
    locs = []
    prev_ner = None  # To keep track of the previous NER tag
    ner_tokens = []  # To collect consecutive tokens
    for index, row2 in sub.iterrows():
        ner = row2['tag']
        if prev_ner == ner:
            ner_tokens.append(row2['token'])
        else:
            # Start of a new NER tag, process the collected tokens
            if prev_ner == 'Person':
                pers.append(' '.join(ner_tokens))
            elif prev_ner == 'Organization':
                orgs.append(' '.join(ner_tokens))
            elif prev_ner == 'Location':
                locs.append(' '.join(ner_tokens))
            ner_tokens = [row2['token']]  # Start a new collection
        prev_ner = ner
    # Process the last collected tokens
    if prev_ner == 'Person':
        pers.append(' '.join(ner_tokens))
    elif prev_ner == 'Organization':
        orgs.append(' '.join(ner_tokens))
    elif prev_ner == 'Location':
        locs.append(' '.join(ner_tokens))

    subP = sub[sub['tag']=='Person']
    tp = list(subP['token'])
    allP = list(set(tp+pers))
    subO = sub[sub['tag']=='Organization']
    to = list(subO['token'])
    allO = list(set(to+orgs))
    subL = sub[sub['tag']=='Location']
    tl = list(subL['token'])
    allL = list(set(tl+locs))
    # print(f"Persons: {len(pers)} : {allP}")
    # print(f"Organizations: {len(orgs)} : {allO}")
    # print(f"Locations: {len(locs)} : {allL}")
    # df2 = df2.append({'id': ID, 'NER_Persons': allP, 'NER_Orgs': allO, 'NER_Loc': allL}, ignore_index=True)
    df2 = pd.concat([df2, pd.DataFrame({'id': [ID], 'NER_Persons': [allP], 'NER_Orgs': [allO], 'NER_Loc': [allL]})], ignore_index=True)
    # break


In [ ]:
df = pd.merge(df, df2, on='id')
df.head(2)

,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized,NER_Persons,NER_Orgs,NER_Loc
0,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,2023-05-31,https://www.bbc.com/urdu/articles/c0j5ze7l39no,پاکستان تحریک انصاف کے چئیرمین عمران خان کے خل...,BBC,پاکستان,عمران خان کی عدالت میں پیشی کا احوال: ’جج تھری...,عمران خان کی عدالت میں پیشی کا احوال جج تھری و...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,عمران خان عدالت پیشی احوال: ’جج تھری ورکنگ ڈیز...,عمران خان عدالت پیشی احوال جج تھری ورکنگ ڈیز پ...,pakistan,NotAssigned,"['عمران', 'عدالت', 'خان', 'ضمانت', 'وکلا', 'مق...",1,1,"[عمران, خان, کی, عدالت, میں, پیشی, کا, احوال, ...","[عارف علوی, عارف, حکنمامہ, عمران, اٹارنی, متعق...","[تحریک انصاف, تحریک, انصاف]","[اسلام, اسلام آباد, آباد, پاکستان]"
1,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,2023-05-30,https://www.bbc.com/urdu/articles/cd1zdj4evl3o,چیئرمین تحریک انصاف عمران خان کے خلاف کم از کم...,BBC,پاکستان,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,نو مئی کے ہنگاموں پر عمران خان کے خلاف کون سی ...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,نو مئی ہنگاموں عمران خان خلاف دفعات تحت مقدمات...,pakistan,NotAssigned,"['دفعات', 'کارکنان', 'درج', 'آر', 'ایف', 'عمرا...",2,1,"[نو, مئی, کے, ہنگاموں, پر, عمران, خان, کے, خلا...","[اسد, چیمہ, فرخ, سعید, اسلم, شاہد, جمشید, عمرا...","[تحریک انصاف, تحریک, انصاف]","[لاہور, گلبرگ, راولپنڈی, پاکستان]"


In [ ]:
df.columns

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc'],
      dtype='object')

# Save Dataset

In [ ]:
df.to_csv(path, index=False)